### **DISCLAIMER**:
##### This notebook was originally developed on the Kaggle website (as a solution to lack of computer power). ***Please restrain yourself from running the following section unless you are also using an online service to run this notebook***.

### Kaggle ONLY section

(modify paths to fit the service you're using)

In [ ]:
!pip install tensorflow
!pip install torch torchvision torchaudio
!pip install openimages

In [ ]:
!mkdir /kaggle/working/guns
!mkdir /kaggle/working/non-guns
!touch /kaggle/working/guns/exclusions.txt
!touch /kaggle/working/non-guns/exclusions.txt

### Imports and Data Gathering

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import preprocess_input
from openimages.download import download_images

##### Downloading the guns data

In [ ]:
# Define the categories you want to download
categories = ["Handgun", "Rifle", "Shotgun"]

# Path to the exclusions file (modify if not working on Kaggle notebook)
exclusions_path = '/kaggle/working/guns/exclusions.txt'

# Download images for each category and store them on created folder (guns)
for category in categories:
    download_images('/kaggle/working/guns', [category], exclusions_path, limit=2000)

##### Downloading the non-guns data

In [ ]:
# Define the categories you want to download
categories = ["Cat", "Dog", "Car"]

# Path to the exclusions file (modify if not working on Kaggle notebook)
exclusions_path = '/kaggle/working/non-guns/exclusions.txt'

# Download images for each category and store them on created folder (non-guns)
for category in categories:
    download_images('/kaggle/working/non-guns/', [category], exclusions_path, limit=2000)

### Model Preprocessing

In [ ]:
# Define function to load and preprocess images
def load_images_from_folder(root, img_size=(224, 224)):
    images = []
    labels = []
    # Map the categories
    label_map = {'handgun': 0, 'rifle': 0, 'shotgun': 0, 'cat': 1, 'dog': 1, 'car': 1}
    # Loop through images and return as array with labels
    for category in os.listdir(root):
        category_path = os.path.join(root, category, "images")
        if os.path.isdir(category_path):
            for img_name in os.listdir(category_path):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(category_path, img_name)
                    image = Image.open(img_path).convert("RGB")
                    image = image.resize(img_size)
                    image = np.array(image)
                    image = preprocess_input(image)
                    images.append(image)
                    labels.append(label_map.get(category, -1))
    return np.array(images), np.array(labels)

In [ ]:
# Load and preprocess datasets
X_guns, y_guns = load_images_from_folder('/kaggle/working/guns')
X_non_guns, y_non_guns = load_images_from_folder('/kaggle/working/non-guns')

In [ ]:
# Combine datasets
X = np.concatenate((X_guns, X_non_guns), axis=0)
y = np.concatenate((y_guns, y_non_guns), axis=0)

In [ ]:
# Ensure the labels are binary: 0 for guns, 1 for non-guns
y = np.where(y == 0, 0, 1)

# One-hot encode labels
y = to_categorical(y, num_classes=2)

In [ ]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024) 

### Our Baseline

In [ ]:
# Count the number of images in each class
guns_count = np.sum(y == 0)
non_guns_count = np.sum(y == 1)

In [ ]:
# Print the class distribution
print(f'Number of Gun Images: {guns_count}')
print(f'Number of Non-Gun Images: {non_guns_count}')

In [ ]:
# Calculate the baseline accuracy (most frequent class)
majority_class_count = max(guns_count, non_guns_count)
total_count = guns_count + non_guns_count
baseline_accuracy = majority_class_count / total_count

In [ ]:
print(f'Baseline Accuracy: {baseline_accuracy:.4f}')

### Creating our model:

In [ ]:
# Load VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the layers except the last few
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [ ]:
# Input layer
inputs = Input(shape=(224, 224, 3))

In [ ]:
# Pass the inputs through the base model
x = base_model(inputs)

In [ ]:
# Add custom layers on top
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(2, activation='softmax')(x)

In [ ]:
# Define the model
model = Model(inputs, outputs)

In [ ]:
# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=[early_stopping]
)

In [ ]:
plt.figure(figsize=(12, 6))

# Plot training & validation loss values
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()

In [ ]:
# Predict and evaluate
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Flatten y_test and y_pred
y_test_flat = y_test.flatten()
y_pred_flat = y_pred.flatten()

# Compute confusion matrix
cm = confusion_matrix(y_test_flat, y_pred_flat)
print("Confusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test_flat, y_pred_flat))

In [ ]:
# Save the trained model
model.save('weapon_class.h5')